## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import Google_Map_API_KEY

# Configure gmaps API key
gmaps.configure(api_key=Google_Map_API_KEY)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Moratuwa,LK,6.7730,79.8816,80.62,75,89,1.32,overcast clouds,2022-02-06 03:31:59
1,1,Mataura,NZ,-46.1927,168.8643,57.47,38,11,5.03,few clouds,2022-02-06 03:31:59
2,2,Ixtapa,MX,20.7000,-105.2000,72.63,73,20,4.61,few clouds,2022-02-06 03:29:44
3,3,Brae,GB,60.3964,-1.3530,39.11,54,90,31.83,overcast clouds,2022-02-06 03:32:00
4,4,Touros,BR,-5.1989,-35.4608,77.90,72,35,9.40,scattered clouds,2022-02-06 03:32:02


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
1,1,Mataura,NZ,-46.1927,168.8643,57.47,38,11,5.03,few clouds,2022-02-06 03:31:59
3,3,Brae,GB,60.3964,-1.3530,39.11,54,90,31.83,overcast clouds,2022-02-06 03:32:00
7,7,Komsomolskiy,UZ,40.4272,71.7189,38.12,32,100,9.22,overcast clouds,2022-02-06 03:32:04
8,8,Matamba,TZ,-8.9833,33.9667,55.56,86,100,3.33,overcast clouds,2022-02-06 03:32:05
19,19,Huilong,CN,31.8111,121.6550,41.99,48,97,9.78,overcast clouds,2022-02-06 03:32:11
20,20,Strezhevoy,RU,60.7333,77.5889,15.78,97,100,4.12,overcast clouds,2022-02-06 03:32:12
21,21,Ushuaia,AR,-54.8000,-68.3000,49.66,57,20,18.41,few clouds,2022-02-06 03:31:28
22,22,Diffa,NE,13.3154,12.6113,59.77,42,0,10.33,clear sky,2022-02-06 03:32:13
23,23,Albany,US,42.6001,-73.9662,10.35,71,5,5.97,clear sky,2022-02-06 03:28:15
32,32,Nikolskoye,RU,59.7035,30.7861,27.72,97,100,4.72,overcast clouds,2022-02-06 03:32:17


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                256
City                   256
Country                256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
Date                   256
dtype: int64

In [6]:
# test_df = preferred_cities_df
# test_df.head()

In [7]:
# test_df = test_df.dropna()
# test_df.count()

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mataura,NZ,57.47,few clouds,-46.1927,168.8643,
3,Brae,GB,39.11,overcast clouds,60.3964,-1.3530,
7,Komsomolskiy,UZ,38.12,overcast clouds,40.4272,71.7189,
8,Matamba,TZ,55.56,overcast clouds,-8.9833,33.9667,
19,Huilong,CN,41.99,overcast clouds,31.8111,121.6550,
20,Strezhevoy,RU,15.78,overcast clouds,60.7333,77.5889,
21,Ushuaia,AR,49.66,few clouds,-54.8000,-68.3000,
22,Diffa,NE,59.77,clear sky,13.3154,12.6113,
23,Albany,US,10.35,clear sky,42.6001,-73.9662,
32,Nikolskoye,RU,27.72,overcast clouds,59.7035,30.7861,


In [10]:
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Google_Map_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
  



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mataura,NZ,57.47,few clouds,-46.1927,168.8643,Ellie's Villa
3,Brae,GB,39.11,overcast clouds,60.3964,-1.3530,Busta House Hotel
7,Komsomolskiy,UZ,38.12,overcast clouds,40.4272,71.7189,Ikathouse
8,Matamba,TZ,55.56,overcast clouds,-8.9833,33.9667,Lena blumen lodge
19,Huilong,CN,41.99,overcast clouds,31.8111,121.6550,Hilton Garden Inn Qidong
20,Strezhevoy,RU,15.78,overcast clouds,60.7333,77.5889,Sibiryachka
21,Ushuaia,AR,49.66,few clouds,-54.8000,-68.3000,Albatross Hotel
22,Diffa,NE,59.77,clear sky,13.3154,12.6113,ProDAF/Diffa
23,Albany,US,10.35,clear sky,42.6001,-73.9662,
32,Nikolskoye,RU,27.72,overcast clouds,59.7035,30.7861,Tourist House - Sablino


In [12]:
import numpy as np

In [13]:
# # test and make sure before actual change, make a test dataframe
# nonfound_df = hotel_df
# nonfound_df.head()
# nonfound_df.count()

In [14]:
# # test and make sure before actual change, make a test dataframe
# emptylist_df = nonfound_df.loc[nonfound_df["Hotel Name"] == '']
# emptylist_df


In [15]:
# # test and make sure before actual change, make a test dataframe
# emptylist_df.loc[emptylist_df["Hotel Name"] == '',["Hotel Name"]] = np.nan
# emptylist_df

In [16]:
# # test and make sure before actual change, make a test dataframe
# emptylist_df = emptylist_df.dropna()
# emptylist_df

In [17]:
# # test and make sure before actual change, make a test dataframe
# nonfound_df.count()

In [18]:
# # test and make sure before actual change, make a test dataframe
# nonfound_df.loc[nonfound_df["Hotel Name"] == '',["Hotel Name"]] = np.nan
# # nonfound_df

In [19]:
# # test and make sure before actual change, make a test dataframe
# nonfound_df = nonfound_df.dropna()
# nonfound_df.head(10)

In [20]:
# 7. Drop the rows where there is no Hotel Name.
# first, search value under "Hotel Name" column, and if is empty ''
# fill Hotel Name that's empty as NaN value
# Then drop the row with NaN values
hotel_df.loc[hotel_df["Hotel Name"] == '',["Hotel Name"]] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mataura,NZ,57.47,few clouds,-46.1927,168.8643,Ellie's Villa
3,Brae,GB,39.11,overcast clouds,60.3964,-1.3530,Busta House Hotel
7,Komsomolskiy,UZ,38.12,overcast clouds,40.4272,71.7189,Ikathouse
8,Matamba,TZ,55.56,overcast clouds,-8.9833,33.9667,Lena blumen lodge
19,Huilong,CN,41.99,overcast clouds,31.8111,121.6550,Hilton Garden Inn Qidong
20,Strezhevoy,RU,15.78,overcast clouds,60.7333,77.5889,Sibiryachka
21,Ushuaia,AR,49.66,few clouds,-54.8000,-68.3000,Albatross Hotel
22,Diffa,NE,59.77,clear sky,13.3154,12.6113,ProDAF/Diffa
23,Albany,US,10.35,clear sky,42.6001,-73.9662,NaN
32,Nikolskoye,RU,27.72,overcast clouds,59.7035,30.7861,Tourist House - Sablino


In [21]:
# drop the row with NaN values
hotel_df = hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mataura,NZ,57.47,few clouds,-46.1927,168.8643,Ellie's Villa
3,Brae,GB,39.11,overcast clouds,60.3964,-1.3530,Busta House Hotel
7,Komsomolskiy,UZ,38.12,overcast clouds,40.4272,71.7189,Ikathouse
8,Matamba,TZ,55.56,overcast clouds,-8.9833,33.9667,Lena blumen lodge
19,Huilong,CN,41.99,overcast clouds,31.8111,121.6550,Hilton Garden Inn Qidong
20,Strezhevoy,RU,15.78,overcast clouds,60.7333,77.5889,Sibiryachka
21,Ushuaia,AR,49.66,few clouds,-54.8000,-68.3000,Albatross Hotel
22,Diffa,NE,59.77,clear sky,13.3154,12.6113,ProDAF/Diffa
32,Nikolskoye,RU,27.72,overcast clouds,59.7035,30.7861,Tourist House - Sablino
33,Biltine,TD,58.96,few clouds,14.5333,20.9167,Hôtel Trois Étoiles de Biltine


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
#fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))